### TODO:





### IDEAS
How is this affecting di
* This could be an ongoing project with a website track how events affect different disorders and suicidal posts
* Build models detecting each subreddit. Then use prob for seeing how anxious, paranoid, etc, each post is. And I think most will become more similar to anxiety as corona virus increases
* predict flu trends (CDC) with healthanxiety posts
* Predict if post is from a certain disorder, is suicidal, pre or post coronavirus,  
* model this with important events: https://towardsdatascience.com/inferring-causality-in-time-series-data-
* How is coronavirus affecting divorce/finance?
* How is health anxiety - social anxiety - mindfulness
* 

In [1]:
# Run this cell to be able to mount GDrive and attach it to the colab so that we can save json outputs
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [2]:
'''
Authors: Daniel M. Low
License: Apache 2.0

'''


import os
import sys
import numpy as np
import pandas as pd
import json
from pprint import pprint
import requests
import time

from datetime import datetime, date, timedelta
import string
from scipy import stats

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
import plotly.express as px
from plotly import graph_objs as go

# sys.path.append('./../../catpro')
# from catpro.preprocessing_text.extract_features import liwc, words, punctuation_count, sentiment_analysis, count_words, tfidf

ModuleNotFoundError: No module named 'plotly'

import plotly.express as px
import plotly.graph_objs as go
import chart_studio.plotly as py

In [ ]:
def date2timestamp(date):
    '''
    "01/12/2011"
    '''
    return int(time.mktime(datetime.strptime(date, "%Y/%m/%d").timetuple()))

def timestamp2date(unix_timestamp):
    '''
    1284101485
    # if you encounter a "year is out of range" error the timestamp
    # may be in milliseconds, try `ts /= 1000` in that case
    print()
    '''
    return datetime.utcfromtimestamp(unix_timestamp).strftime('%Y-%m-%d %H:%M:%S')


def url2json(url, output_filepath):
    solditems = requests.get(url) # (your url)
    data = solditems.json()
    with open(output_filepath+'.json', 'w') as f:
        json.dump(data, f)
    return

def json2list(json_filepath):
    with open(json_filepath) as f:
        data = json.load(f)['data']
    return data




def list_of_days(start_date, end_date):
    '''
    start_date = date(2019, 1, 19)   # start date
    end_date = date(2019, 3, 22)   # end date
    '''
    delta = end_date - start_date       # as timedelta
    days = []
    for i in range(delta.days + 1):
        day = start_date + timedelta(days=i)
        days.append(str(day).replace('-', '/'))
    return days


In [ ]:
# input y variable across time


def plot_across_time(subreddits, x = None, y_col = '', y_col_div_by = '', zscore=False, ylabel='Average negative sentiments',filter_small = True, small_value = 150, plot_raw = True, 
                     plot_line = True, alpha = 0.3, plot_covid_curves = False):
    '''
    y_col: {'all_words', ''}
    y_col_div_by: {'total_posts'}
    small_vale = if you want to include small posts, set to 0. 150 shizophrenia
    '''
    sns.set(style='white', rc={'figure.figsize':(10,8)}, font_scale =1.5)
    cmap = cm.Set3 # tab20c Set3
    cmap = cm.jet(np.linspace(0,1,len(subreddits)+1))
        
    # Load each rubreddit 
    for i,subreddit in enumerate(subreddits):

        df_subreddit = pd.read_csv(output_dir+'features_{}_0.csv'.format(subreddit))
        # Some just have avg 10 posts per time segment, so not worth plotting        
        total_posts = df_subreddit[['subreddit', 'date']].groupby(['date']).agg(['count']).values
        if np.mean(total_posts)  < small_value:
            continue
        # Load y
        if y_col == 'total_posts':
            y = total_posts.copy()
        else:
            y = df_subreddit[[y_col, 'date']].groupby(['date']).agg(['sum']).values
        if y_col_div_by == 'total_posts':
            y = y/total_posts
        
        y = [n[0] for n in y]
        if zscore:
            if len(np.unique(y)) == 1:
                # By mistake I downloaded every 3 days and some subreddits max at 1000, so zscore of same values (1000,1000,..., 1000) is not possible                 
                continue
            else:
                y = stats.zscore(y)

        # Load x, is the same for all         
        if x == None:
            x = list(set(df_subreddit['date'].values))
            try: x.remove(np.nan)
            except: pass
            x.sort()
        x_num =range(len(x))  
        if plot_raw:
            plt.plot(x,y, alpha=alpha,color=cmap[i])
        if plot_line:
            m, b = np.polyfit(x_num, y, 1)
            plt.plot(x, m*x_num + b, color=cmap[i], alpha=1,label='r/'+subreddit)
        else:
            plt.plot(x,y, alpha=1,color=cmap[i],label='r/'+subreddit)
    
    plt.xticks(ticks=range(len(list(x))), labels=list(x),rotation=90)
    
    # Plot covid confirmed cases 
    if plot_covid_curves:
        for y_country, country,color in zip(y_countries, countries, ['k','gray','darkgray']):
            plt.plot(x,y_country, alpha=1,color=color, label='Confirmed cases - {}'.format(country))
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    if zscore:
        ylabel += ' (z-score)'
    plt.ylabel(ylabel)
    return



    

In [ ]:

def boxplot_across_time(subreddit, x = None, y_col = '', y_col_div_by = '', zscore=False, ylabel='Average negative sentiments',filter_small = True, small_value = 150, plot_raw = True, 
                     plot_line = True, alpha = 0.3, plot_covid_curves = False):
    '''
    y_col: {'all_words', ''}
    y_col_div_by: {'total_posts'}
    small_vale = if you want to include small posts, set to 0. 150 shizophrenia
    '''
    sns.set(style='white', rc={'figure.figsize':(10,8)}, font_scale =1.5)
    cmap = cm.Set3 # tab20c Set3
    cmap = cm.jet(np.linspace(0,1,len(subreddits)+1))
        
    # Load each rubreddit 
    for i,subreddit in enumerate(subreddits):

        df_subreddit = pd.read_csv(output_dir+'features_{}_0.csv'.format(subreddit))
        # Some just have avg 10 posts per time segment, so not worth plotting        
        total_posts = df_subreddit[['subreddit', 'date']].groupby(['date']).agg(['count']).values
        if np.mean(total_posts)  < small_value:
            continue
        # Load y
        y = df_subreddit[[y_col, 'date']].groupby(['date']).agg(['sum']).values
        if y_col_div_by == 'total_posts':
            y = y/total_posts
        
        y = [n[0] for n in y]
        if zscore:
            if len(np.unique(y)) == 1:
                # By mistake I downloaded every 3 days and some subreddits max at 1000, so zscore of same values (1000,1000,..., 1000) is not possible                 
                continue
            else:
                y = stats.zscore(y)

        # Load x, is the same for all         
        if x == None:
            x = list(set(df_subreddit['date'].values))
        x_num =range(len(x))  
        if plot_raw:
            plt.plot(x,y, alpha=alpha,color=cmap[i])
        if plot_line:
            m, b = np.polyfit(x_num, y, 1)
            plt.plot(x, m*x_num + b, color=cmap[i], alpha=1,label='r/'+subreddit)
        else:
            plt.plot(x,y, alpha=1,color=cmap[i],label='r/'+subreddit)
    
    plt.xticks(ticks=range(len(list(x))), labels=list(x),rotation=90)
    
    # Plot covid confirmed cases 
    if plot_covid_curves:
        for y_country, country,color in zip(y_countries, countries, ['k','gray','darkgray']):
            plt.plot(x,y_country, alpha=1,color=color, label='Confirmed cases - {}'.format(country))
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    if zscore:
        ylabel += ' (z-score)'
    plt.ylabel(ylabel)
    return



    

In [ ]:
# Load data
# output_dir = './../../datum/reddit/input/'

# potential output dir if drive has been mounted:
output_dir = '/content/drive/My Drive/reddit_json/'
    

In [ ]:
df = pd.read_csv(output_dir+'features_depression_0.csv')
df.shape

In [ ]:
# Search reddit
# Search unix code https://www.unixtimestamp.com/index.php
# Timeline: https://www.nytimes.com/article/coronavirus-timeline.html


In [ ]:
# Anxiety corona virus
# JAN. 20 Other countries, including the United States, confirmed cases.
"I know this may sound silly, but I\u2019m scared of the coronavirus. A lot of people are saying it\u2019s not a big deal and that I\u2019ll be safe as long as I\u2019m healthy. I know all of this but I\u2019m more scared of the fact that this will turn into a pandemic."

# Other days:
# JAN. 30 The W.H.O. declared a global health emergency
# The World Health Organization on Feb. 11 proposed an official name for the disease the virus coronavirus causes: Covid-19
# Feb 23 Italy sees major surge in coronavirus cases and officials lock down towns.
subreddit = 'healthanxiety'
date_start = '2020/03/24'
date_end = '2020/03/30'

start = date2timestamp(date_start)
end = date2timestamp(date_end)
size = 1000000000
url = 'https://api.pushshift.io/reddit/search/submission/?subreddit={}&sort=desc&sort_type=created_utc&after={}&before={}&size={}'.format(subreddit,start, end, size)
print(url)
# filename = 'scrape_reddit'
# filename = filename+'_{}_{}'.format(date_start.replace('/', ''), date_end.replace('/', ''))


# Comment so it won't download

# url2json(url, output_dir+filename)
# anxiety_covid = json2list(output_dir+filename+'.json')

In [ ]:
timestamp2date(1579395379)

In [0]:
def scrape_reddit(output_dir, subreddit, timeframe, date_start, date_end, size = 1000):
    '''
    size = {1,1000} #amount of posts
    '''
    start = date2timestamp(date_start)
    end = date2timestamp(date_end)
    url = 'https://api.pushshift.io/reddit/search/submission/?subreddit={}&sort=desc&sort_type=created_utc&after={}&before={}&size={}'.format(subreddit,start, end, size)
    filename = '{}_{}_{}_{}'.format(subreddit, timeframe, date_start.replace('/', ''), date_end.replace('/', ''))
    url2json(url, output_dir+filename)
    posts = json2list(output_dir+filename+'.json')
    posts_filtered = []
    for post in posts:
        post_filtered = {}
        for key in post.keys():
            post_filtered[key]=post.get(key)
        posts_filtered.append(post_filtered)
    # np.savez_compressed(input_dir+filename+'.npz',posts_filtered)
#     os.remove(output_dir+filename+'.json')
    return filename
        


In [0]:
date2timestamp(date_start)

In [14]:
# # Commented so it won't run on accident





# start = date2timestamp(date_start)
# end = date2timestamp(date_end)


days = list_of_days(date(2020,3,27), date(2020,3,30))
combinations = [['depression', 'post']]
size = 1000

# Scrape: takes a while
for subreddit, timeframe in combinations:
    for i in range(len(days))[::3]:
        try:
            date_start = days[i]
            date_end = days[i+3]
            i=+ 3
        except:
            break
        print(i)
        print(date_start)
#         print(subreddit, timeframe, date_start, date_end)
        filename = scrape_reddit(output_dir, subreddit, timeframe, date_start, date_end, size = size)
#         posts_filtered = np.load(output_dir+filename+'.npy')

3
2020/03/27


In [0]:
scraped = ['EDAnonymous',
 'addiction',
 'adhd',
 'alcoholism',
 'anxiety',
 'askdocs',
 'askscience',
 'atheism',
 'bipolarreddit',
 'books',
 'bpd',
 'buddhism',
 'christianity',
 'depersonalization',
 'depression',
 'dpdr',
 'eatingdisorders',
 'features',
 'feelgood',
 'fitness',
 'frugal',
 'healthanxiety',
 'jokes',
 'legaladvice',
 'lifeprotips',
 'meditation',
 'mentalhealth',
 'mentalillness',
 'mindfulness',
 'nostupidquestions',
 'paranoia',
 'parenting',
 'personalfinance',
 'psychosis',
 'psychoticreddit',
 'ptsd',
 'randomkindness',
 'relationships',
 'schizophrenia',
 'showerthoughts',
 'socialanxiety',
 'stopselfharm',
 'suicidewatch',
 'talesfromcallcenters',
 'talesfromretail',
 'talesfromtechsupport',
 'teaching',
 'theoryofreddit',
 'writing',
 'writingprompts']




In [0]:

subreddits = ['mentalhealth',
 'socialanxiety',
 'buddhism',
 'depersonalization',
 'randomkindness',
 'ptsd',
 'alcoholism',
 'depression',
 'mindfulness',
 'mentalillness',
 'psychosis',
 'suicidewatch',
 'atheism',
 'bpd',
 'fitness',
 'adhd',
 'stopselfharm',
 'parenting',
 'meditation',
 'healthanxiety',
 'teaching',
 'relationships',
 'schizophrenia',
 'personalfinance',
 'EDAnonymous',
 'jokes',
 'addiction',
 'anxiety',
 'bipolarreddit',
 'paranoia',
 'christianity',
 'legaladvice']


In [0]:
# # Commented so it won't run on accident


# # Load scraped files and Extract features WITHOUT SUMMERIZING

# # features.append([subreddit, timeframe, date_range, total_posts] + [a0,a1,b]+c+c1+c2+d)        
# columns = ['subreddit','timeframe','date','posts','posts_raw', 'all_words', 'covid_total', 'covid_boolean', 'suicide_total','suicide_bool', ]
# sentiment_cols = ['sent_neg','sent_neu', 'sent_pos', 'sent_compound']
# # liwc_cols = pd.read_csv(liwc_dir + 'categories.txt', index_col=0)['0'].tolist()
# columns = columns+sentiment_cols


# covid_words = ['corona','virus','coronavirus', 'pandemic', 'epidemic', 'quarantine', 'covid', 'covid19']
# suicide_words = ['end it', "take it anymore", 'suicide', 'I will overdose','I want to overdose', 'shoot myself', 'hang myself', 'kill myself', "hurt myself", "cut myself",  
#                 'will miss me', 'if I live or die', 'hopeless'] #Look at suicide note corpus for more

# Adam sent these:
# suicide_thoughts = ['suicidal', 'to die', 'death', 'kill', 'kill myself', 'want to die', 'suicide', 'to end', 'wanna die', 'wanna kill', 'commit suicide', 'killing myself', 'killing', 'to kill', 'die', 'dying', 'alive', 'live anymore', 'goodbye', 'this world', 'earth', 'end my', 'my life', 'life anymore', 'live anymore', "i'm done" "i'm just done", 'so done', 'end', 'am done', 'overdose', "I'm done", 'end tonight', 'ends tonight', 'I wish', 'just wish', 'sleep forever', 'never wake', 'commit', 'with life', 'wake up', 'shoot me', 'hang myself', 'a bridge', 'to live', 'wanna live', 'dead', 'be dead', 'just die', 'gun']



# timeframes = ['post']
# combinations = []
# for sr in subreddits:
#     for t in timeframes:
#         combinations.append([sr, t])

# days_all = [list_of_days(date(2019, 1, 19), date(2019, 3, 22)), list_of_days(date(2020, 1, 19), date(2020, 3, 22)),] 


# for subreddit, timeframe in combinations:
#     features_df = pd.DataFrame()
#     print(subreddit)
#     features = []
#     if timeframe == 'pre':
#         days = days_all[0]
#         year = '2019'
#     else:
#         days = days_all[1]
#         year = '2020'
#     # For each timepoint 

#     for i in range(len(days))[::3]:
#         try:
#             date_start = days[i]
#             date_end = days[i+3]
#             i=+ 3
#         except:
#             break
#         filename = '{}_{}_{}_{}'.format(subreddit, timeframe, date_start.replace('/', ''), date_end.replace('/', ''))
#         # Load 1 timepoint         
#         posts = np.load(output_dir+filename+'.npz')['arr_0'] 


#         posts_raw = [(post['title']+' '+post['selftext']).replace('. ', '.\n') for post in posts if post['selftext']!=None and post['title']!=None]
#         posts = [' '.join((post['title']+' '+post['selftext']).split()) for post in posts if post['selftext']!=None and post['title']!=None]
#         date_range = date_start.replace(year+'/', '')+'-'+date_end.replace(year+'/', '')
#         # Features         
#         word_count_all = [words(post) for post in posts_clean] # count all words
#         word_count_covid = [count_words(post, covid_words) for post in posts] # count covid words
#         word_count_covid_total = [n[0] for n in word_count_covid] #total
#         word_count_covid_bool = [n[1] for n in word_count_covid] #appears at least once in post    
#         word_count_suicide = [count_words(post, suicide_words) for post in posts] # count suicide
#         word_count_suicide_total = [n[0] for n in word_count_suicide] #total
#         word_count_suicide_bool = [n[1] for n in word_count_suicide] #appears at least once in post    
        
#         sent = [list(sentiment_analysis(post).values()) for post in posts]
#         sent0 = [n[0] for n in sent]
#         sent1 = [n[1] for n in sent]
#         sent2 = [n[2] for n in sent]
#         sent3 = [n[3] for n in sent]

#         df_day_i = pd.DataFrame([
#             [subreddit]*len(posts), 
#             [timeframe]*len(posts),
#             [date_range]*len(posts),
#             posts, posts_raw, word_count_all, 
#             word_count_covid_total, word_count_covid_bool,  word_count_suicide_total, word_count_suicide_bool,
#             sent0,sent1,sent2,sent3,
#         ]).T
#         features_df = pd.concat([features_df, df_day_i])
        
        
# #         d = [liwc(liwc_dir,post) for post in posts] #LIWC
# #         d = list(pd.DataFrame(d).mean(axis=0))
# #         features.append([subreddit, timeframe, date_range, total_posts] + [a0,a1,b]+c0+c1+d)            
#     features_df.columns=columns
#     features_df.to_csv(output_dir+'features_{}_0.csv'.format(subreddit), index=0)

In [0]:
# Obtain curves for confirmed cases
countries = ['China', 'US', 'Italy']
scale = 500000

y_countries = []

for country in countries:
    confirmed = pd.read_csv(output_dir+'time_series_covid_19_confirmed.csv')
    confirmed_country = confirmed[confirmed['Country/Region']==country]
    y_country = confirmed_country.iloc[:,4:].sum().values
    y_country = list(y_country[::3]/scale)
    # x_country = list(country_country.iloc[:,4:].sum().index)
    # x_country = [0]+list(x_country[::3])
    y_countries.append(y_country)
    
len(y_countries[0])

In [0]:
# # How covid is affecting people who might be suicidal
# fig = px.scatter(features_df, x="date", y="covid_total", hover_name = 'posts_raw')
# fig.show()

# # Plot more than one
# # trace0 = go.Scatter(x=date, y=features_df.suicide_total.values)#, hover_name = features_df.posts_raw)
# # trace1 = go.Scatter(x=date, y=features_df.covid_total.values)#, hover_name = features_df.posts_raw)
# # data = [trace0, trace1]
# # py.iplot(data, filename='scatter-mode')



In [0]:
# # Concat extracted features into 1 df:
# features_df = pd.read_csv(output_dir+'features_{}.csv'.format('mindfulness'))
# print(features_df.shape)
# for subreddit in features_extracted:
#     if subreddit == 'mindfulness':
#         continue
#     features = pd.read_csv(output_dir+'features_{}.csv'.format(subreddit))
#     features_df = pd.concat([features_df,features])
    

In [0]:
# subredditsx = ['anxiety', 'healthanxiety']
plot_across_time(subreddits,y_col = 'covid_boolean', y_col_div_by = 'total_posts',
                  ylabel='Proportion of subReddit posts about COVID19',filter_small = True, small_value = 150, 
                  plot_raw = True,  plot_line = False, alpha = 0.3, plot_covid_curves = True, zscore = False)



In [0]:
### Hypochondriac symptoms predict overall concern 1.5 months before. 
### All of Reddit converges on the same topic

In [0]:
'''
# Early:
01/19: {'created_utc': 1579618862, 'num_comments': 7, 'title': 'The breaking news stories about the coronavirus is making me really nervous and I’m not sure what to do?', 'selftext': 'I’ve been seeing a lot of news stories today about an incurable, lethal disease originating in China spreading to other countries. I know the media often over exaggerates things like this, but WHO is debating whether or not to declare this an international emergency. Nothing triggers my anxiety worse than things like this, if anyone has any advice on how to cope/can inform me on it I would really appreciate it!', 'score': 1},
01/19: {'created_utc': 1579543321, 'num_comments': 1, 'title': 'Wuhan Virus anxiety', 'selftext': "Right off the bat, i'm not chinese or live anywhere in the eastern part of the globe. But i am a massive hypochondriac and terrified of a global outbreak.  \n\n\nThis morning, i saw an article showing a video of airline passengers being screened by medical personal in hazmat suits. With the article spewing the whole sensationalized clickbait crap about an apocalyptic global epidemic.  \n\n\nAnd after seeing it i quickly spiraled into stress and anxiety. Even though i was able to avoid a full on panic attack, this article bothered me to the point that ll i've been thinking about is an outbreak for the rest of the day just by reading a news article for a few minutes.  \n\n\nI hope everybody in the east can stay safe and healthy, the worst i've had today is nearly having a panic attack. Can't imagine how worrying it is over there.", 'score': 1},
01/25: {'created_utc': 1580175419, 'num_comments': 3, 'title': 'so so so scared', 'selftext': 'this corona virus has me absolutely terrified. i know there\'s all these "low mortality" things, but i read to far into the conspiracies and the videos of chinese hospitals and doctors crying for help. \n\nive washed and sanitized my hands to the point they\'re cracked and bleeding. i won\'t leave the house anymore unless its 100% necessary.and even when i do i don\'t touch ANYTHING\n\nim so scared, breaking down crying, i can\'t even eat anymore. nothing the news or cdc says can calm me down because i am fully convinced china is lying and that this could even be a bioweapon. i heard it\'s mutated already and 300,000-1,000,000 people fled china possibly infected. i feel doomed'}



# Suicidality
03/16:        {'created_utc': 1584584727, 'num_comments': 0, 'title': 'I feel like a scared child but I have no parents to run to for safety', 'selftext': "I'm far from being actually alone, I don't even live alone, yet right now with the virus, the economic status, and the high stress everyone is feeling is making me feel like my bones are trying to crawl out of my flesh. I want it to stop, I want to scream and cry, I want to hide and honestly just fucking die so I don't have to live with any of these feelings anymore. I work minimum wage, I'm saving for the house I'm renting in May, but if I lose my job or can't work because of Covid-19 I am absolutely fucked. I am terrified and I feel pinned down by this overwhelming feeling it's all going to get so so much worse...Part of me wishes the world would just end. No worried then. No stress. I'm dreading my state saying we must stay home because my father is bipolar and refuses to go to therapy or get medication so I don't feel safe outside of my room and most of my social interactions outside of work come in arguments with him because he can never be wrong. He has to be right. I wish it would all just freeze or disappear.", 'score': 1},
# Works at hospital
03/16:        {'created_utc': 1584581971, 'num_comments': 0, 'title': 'Feeling overwhelmed about corona and working at the hospital', 'selftext': "I live in New York and work in a hospital. We were short on supplies even before the virus really became a thing here. We don't have masks to give patients. We are not allowed to wear masks ourself. We have been mandated to work and even been told to continue coming to work even if we have had contact with a confirmed positive. We just have to take our temperature daily &amp; we are only to wear a mask if we have active respiratory symptoms.\n\nI have constant shortness of breath and chest pains and pressure due to anxiety which has me constantly stressed. My throat is super dry and idk why. Dry throat and stress leaves you more susceptible to infection in addition to working in a war zone with no protection daily.\n\nI feel so anxious and overwhelmed. I was finally getting my panic attacks sorta under control by going out mor and starting therapy. That's all on hold for the indefinite future. Im constantly afraid of having another panic attack or being in the position where I feel like I'm actually dying and unable to see a provider due to the system being overwhelmed.\n\nThe only positive thing I have going on right now is that its about to be spring and I can garden while social distancing.", 'score': 1},
# Buying guns
03/16:        {'created_utc': 1584581413, 'num_comments': 24, 'title': 'Are we going to all die and go into an apocalypse? The media is starting to freak me out because of the Corona Virus', 'selftext': "I keep on seeing the Corona Virus everywhere now I hear people are buying guns then people are talking about Germany and how bad it is there. I thought that America was handling it very well. And we are on top of it? I'm freaking out. Am I overthinking between all the youtube posts I'm buggin.", 'score': 0},

03/16:        {'created_utc': 1584574890, 'num_comments': 1, 'title': 'My mom is driving herself crazy because of the COVID-19', 'selftext': 'My mom never have suffer from anxiety or other mental illness like OCD, etc. \nSince the COVID-19, we steam and sanitize our house all the day with the steamer machine, we use other product like Clorox or Javex, if she touches something in the house she wash her hand, I suffer from anxiety and it doesn’t help at all! I feel stress will kill our brain before the COVID-19, I’m tired, really tired, I want just this stupid virus to end and to go out. All we do is talking about this virus, the media is not helping at all... my head is gonna explode.', 'score': 3},
# GAD + Isolation  
      {'created_utc': 1584570369, 'num_comments': 0, 'title': 'Being single and living alone is absolutely brutal right now.', 'selftext': 'I live with generalized anxiety disorder and already have major anxiety regarding my health. Had the flu end of January pretty bad and then hives post flu- I’m miserable and was finally getting back on track and then this.\n\n\nI am terrified. Just being alone all day. And can’t see my parents or friends. Or brother. I have been so anxious through all of this for many reasons. \n\nfinally got the work from home okay on Monday but now being told by higher ups where I work (a health science college at a major uni) that someone needs to be in my office during the day, though it’s not essential. I hate the idea of going to my office which is connected to the main hospital in my city.\n\nAnyway. I really could use some people to talk to as I don’t have that many friends and that’s what I need now. I need some more connections and chances to get to other people and distract from the virus.\n\nThanks', 'score': 0},

'''

In [0]:
subreddits2 = ['relationships','mindfulness', 'healthanxiety', 'anxiety', 'suicidewatch', 'depression', 'ptsd', 'adhd', 
              'bpd', 'paranoia', 'socialanxiety', 'schizophrenia', 'bipolarreddit', 'alcoholism', 'addiction',]







### Social anxiety: less posts, more words on average per post

In [0]:
plot_across_time(subreddits2, y_col = 'total_posts', y_col_div_by = False, zscore = False, 
                 ylabel='Total posts',
                 small_value = 150, plot_raw = True, 
                     plot_line = True, alpha = 0.1, plot_covid_curves = False)

In [0]:
plot_across_time(subreddits2, y_col = 'total_posts', y_col_div_by = False, zscore = True, 
                 ylabel='Total posts',
                 small_value = 150, plot_raw = True, 
                     plot_line = True, alpha = 0.1, plot_covid_curves = False)


In [0]:
plot_across_time(subreddits2, y_col = 'all_words', y_col_div_by = 'total_posts', 
                 ylabel='Average words per post',
                 small_value = 150, plot_raw = True, zscore = True,
                     plot_line = True, alpha = 0.3, plot_covid_curves = False)

# Sentiment analysis: social anxiety getting less negative!

In [0]:
plot_across_time(subreddits, y_col = 'sent_neg', y_col_div_by = False, 
                 ylabel='Average negative sentiments',zscore = True,
                 filter_small = True, small_value = 200, plot_raw = False, 
                     plot_line = True, alpha = 0.1, plot_covid_curves = False)


In [0]:

# input y variable across time



In [0]:
subreddit = 'socialanxiety'
y_col = 'sent_neg'
df_subreddit = pd.read_csv(output_dir+'features_{}_0.csv'.format(subreddit))

# Load data
xs = list(set(df_subreddit.date.values))
try: xs.remove(np.nan)
except: pass
xs.sort()
ys = []
cs = []
for day in xs:
    df_subreddit_day = df_subreddit[df_subreddit.date == day]
    y = df_subreddit_day[y_col].values
    ys.append(y)
    median_value = int(np.median(y))
    median_max = np.percentile(y,90)
    color = median_max*10**2
    color = np.max(y)*10**2
    cs.append('hsl({}%,50%,50%)'.format(color))


# plot
fig = go.Figure(data=[go.Box(
    y=ys[i],
    marker_color=cs[i],
    name = xs[i],
    ) for i in range(len(xs))])

# format the layout
fig.layout.update(
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=True,   ticktext = xs),
    yaxis=dict(zeroline=False, gridcolor='white'),
    paper_bgcolor='rgb(233,233,233)',
    plot_bgcolor='rgb(233,233,233)',
    showlegend=False,
    title=y_col+' as COVID-19 grows'
)

fig.show()

In [0]:
subreddit = 'socialanxiety'
df_subreddit = pd.read_csv(output_dir+'features_{}_0.csv'.format(subreddit))
fig = px.scatter(df_subreddit, x="date", y="sent_neg", color="suicide_bool",hover_name='posts',
                 title=subreddit)

fig.show()

### Suicidality

In [0]:
subreddit = 'suicidewatch'
df_subreddit = pd.read_csv(output_dir+'features_{}_0.csv'.format(subreddit))
fig = px.scatter(df_subreddit, x="date", y="sent_neg", color="sent_neg",hover_name='posts',
                 title="Numeric 'size' values mean continous color")

fig.show()

In [0]:
subreddit = 'suicidewatch'
df_subreddit = pd.read_csv(output_dir+'features_{}_0.csv'.format(subreddit))
fig = px.scatter(df_subreddit, x="date", y="sent_neg", color="covid_boolean",hover_name='posts',
                 title=subreddit+' with COVID-19 related posts in yellow')

fig.show()

In [0]:
subreddit = 'socialanxiety'
df_subreddit = pd.read_csv(output_dir+'features_{}_0.csv'.format(subreddit))
fig = px.scatter(df_subreddit, x="date", y="sent_neg", color="suicide_bool",hover_name='posts',
                 title=subreddit+' with suicidal related posts in yellow')

fig.show()




In [0]:
# The fact that it may be affecting ED more or certain disorders more could be useful
# How personal finance may be connected to mental could be studied: If personal finance become more similar to some disorder (tfidf))
# Target at risk people who post regularly and stopped posting? 



# TFIDF

In [0]:
# reload(extract_features)

In [0]:
# Create a corpus to build the matrix
posts_all = []
for sr in subreddits2:
    df_subreddit = pd.read_csv(output_dir+'features_{}_0.csv'.format(sr))
    posts = df_subreddit.posts.values
    posts_all.append(posts)
    

corpus = [n for i in posts_all for n in i]
# remove nan
corpus = [n for n in corpus if isinstance(n,str)]

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(lowercase=True, ngram_range=(1,3), stop_words='english',
                        max_features=512, min_df=2, max_df=0.8)



# Build matrix
vectorizer.fit_transform(corpus)

In [0]:
# Comment so it won't run



In [0]:
feature_array = np.array(vectorizer.get_feature_names())
n = 15

split = 0.9



for sr in subreddits2:
    df_subreddit = pd.read_csv(output_dir+'features_{}_0.csv'.format(sr))
    posts = df_subreddit.posts.values
    posts = [n for n in posts if isinstance(n,str)]
    split_i = int(len(posts)*split)
    posts_before = posts[:split_i]
    posts_after = posts[split_i:]
    posts2 = [posts_before, posts_after]
    posts_names = ['before', 'after']
    for i in range(2):
        response = vectorizer.transform(posts2[i])
        tfidf_sorting = np.argsort(response.toarray()).flatten()[::-1]
        top_n = feature_array[tfidf_sorting][:n]
        print('\n====={}==={}'.format(sr, posts_names[i]))
        print(top_n)
    print('\n\n===============')

In [0]:
# # Alternative method
# def tfidf(X_train_sentences = posts, lower_case = True, ngram_range = (1,3), max_features=512, min_df=2, max_df=0.8, model = 'svm'):
#     sw = stopwords.words('english')
#     vectorizer = TfidfVectorizer(lowercase=lower_case, ngram_range=ngram_range, stop_words=sw,max_features=max_features, min_df=min_df, max_df=max_df)
#     train_vectors = vectorizer.fit_transform(X_train_sentences).toarray()
#     feature_names = vectorizer.get_feature_names()
#     feature_names = ['tfidf_'+n for n in feature_names]
#     return train_vectors, feature_names
    
# X_test_sentences = False
# vectors, feature_names = tfidf(X_train_sentences = posts_all_days, lower_case = True, ngram_range = (1,3), max_features=512, min_df=2, max_df=0.8, model = 'svm')
    